In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np

gpus = tf.config.experimental.list_physical_devices("GPU")

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
import numpy as np

def make_dataset(n_samples, n_features, n_targets):
    x = np.random.randn(n_samples, n_features)
    y = np.random.randn(n_samples, n_targets)
    return x, y

x, y = make_dataset(1010, 1, 1)

def segment_dataset(x, y, window_size):
    x_segments = []
    y_segments = []

    for i in range(len(x) - window_size):
        x_segments.append(x[i:i+window_size])
        y_segments.append(y[i:i+window_size])

    return np.array(x_segments), np.array(y_segments)

x, y = segment_dataset(x, y, 10)

print(x.shape, y.shape)
print(x[0], "\n\n", y[0])

input_dim = x.shape[1:]
print(input_dim)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

In [ ]:
def build_classifier(input_dim):
    input_layer = layers.Input(shape=input_dim)
    net = layers.Dense(128, activation="relu")(input_layer)
    net = layers.BatchNormalization()(net)
    net = layers.Dense(1280, activation="relu")(net)
    net = layers.BatchNormalization()(net)
    net = layers.Dropout(0.25)(net)
    net = layers.Dense(12800, activation="relu")(net)
    net = layers.BatchNormalization()(net)
    net = layers.Dense(1280, activation="relu")(net)
    net = layers.BatchNormalization()(net)
    net = layers.Dropout(0.25)(net)
    output_layer = layers.Dense(1, activation="sigmoid")(net)

    model = Model(input_layer, output_layer)
    model.summary()

    return model

In [ ]:
batch_size = 32
learning_rate = 0.001
epochs = 1000000

# Build classifier
classifier = build_classifier(input_dim=input_dim)

# Compile classifier
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Train classifier
classifier.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

# Evaluate classifier
results = classifier.evaluate(x_test, y_test)

print("Loss: %.2f" % results[0])
print("Accuracy: %.2f%%" % (results[1] * 100))